In [ ]:
pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=50d98c8138e81433093a4408912ccf40dc7d78a3dca65467225ee4c0047b90f3
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install flirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import seaborn as sns
import flirt.reader.empatica
import flirt.with_
import matplotlib.pyplot as plt
import matplotlib as mpl
# import latex
import os
import csv
from glob import glob
import copy
import random
import pickle
import numpy as np
from scipy import stats
import scipy.io.wavfile
import scipy.signal
import pingouin as pg
import pandas as pd

def cliffs_delta(lst1, lst2):
    """Calculates Cliff's Delta, a non-parametric measure of effect size for two samples."""
    n1 = len(lst1)
    n2 = len(lst2)
    lst1 = sorted(lst1)
    lst2 = sorted(lst2)
    i, j = 0, 0
    delta = 0
    while i < n1 and j < n2:
        if lst1[i] <= lst2[j]:
            delta += j
            i += 1
        else:
            j += 1
    delta = delta / (n1 * n2)
    return delta

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


In [ ]:
!unzip "/content/Physiological Signal.zip" -d "/content/"
!unzip "/content/Physiological Signal/Dirt Rally.zip" -d "/content/"
!unzip "/content/Physiological Signal/Warthunder.zip" -d "/content/"
!unzip "/content/Physiological Signal/Minecraft.zip" -d "/content/"
!unzip "/content/Physiological Signal/Phasmophobia.zip" -d "/content/"

#Heart Rate

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
hr = [[[], [], [], []], [[], [], [], []]]
hr_bs = [[], [], [], []]
hr_bs_std=[[], [], [], []]
i=-1
for path in paths:
  i+=1
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    hr[0][i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    hr[1][i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
  for participant in glob(path + "/*/Baseline", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    # print(p)
    hr_bs[i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
    hr_bs_std[i].append(np.std(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())

In [ ]:
hr_norm = [[[], [], [], []], [[], [], [], []]]
for i in range(4):
  # print(i)
  hr_norm[0][i] = (np.array(hr[0][i]) - np.array(hr_bs[i]))
  hr_norm[1][i] = (np.array(hr[1][i]) - np.array(hr_bs[i]))
# print(hr_norm[0])

##Mean Values

In [ ]:
Games = ['LVHA', 'HVLA', 'LVLA', 'HVHA']
Type = ['FS', 'VR']

arr=[]
i=0
for x in hr:
  tmp=[]
  j=0
  # print(len(x))
  for y in x:
    df = pd.DataFrame(y, columns = ["Heart Rate"])
    df["CMA Quadrant"] = Games[j]
    df["Type"] = Type[i]
    tmp.append(df)
    j+=1
  arr.append(pd.concat(tmp))
  i+=1

df = pd.concat(arr, ignore_index = True)
print(df)
df.to_csv("HR_Boxplot.csv")

    Heart Rate CMA Quadrant Type
0    85.312029         LVHA   FS
1    79.074062         LVHA   FS
2    84.249643         LVHA   FS
3    80.658842         LVHA   FS
4    75.991310         LVHA   FS
..         ...          ...  ...
61   86.000446         HVHA   VR
62  101.441521         HVHA   VR
63   88.675596         HVHA   VR
64   83.479615         HVHA   VR
65   95.072468         HVHA   VR

[66 rows x 3 columns]


In [ ]:
pc = df.query('Type=="FS"')["Heart Rate"].tolist()
vr = df.query('Type=="VR"')["Heart Rate"].tolist()

print("HR")
print(f"FS: mu: {np.mean(pc):.2f}, sigma: {np.std(pc):.2f}")
print(f"VR: mu: {np.mean(vr):.2f}, sigma: {np.std(vr):.2f}")

HR
FS: mu: 77.27, sigma: 9.92
VR: mu: 81.52, sigma: 10.71


##Comparing HR

In [ ]:
Games = ['LVHA', 'HVLA', 'LVLA', 'HVHA']
Type = ['FS', 'VR']

arr=[]
i=0
for x in hr_norm:
  tmp=[]
  j=0
  # print(len(x))
  for y in x:
    df = pd.DataFrame(y, columns = ["Normalised HR"])
    df["Type"] = Type[i]
    tmp.append(df)
    j+=1
  # print(j)
  arr.append(pd.concat(tmp))
  i+=1
# print(i)

df = pd.concat(arr, ignore_index = True)
print(df)
df.to_csv("HR.csv")

    Normalised HR Type
0       -4.275331   FS
1       -2.864298   FS
2       -8.306594   FS
3      -17.661158   FS
4        0.000000   FS
..            ...  ...
61     -14.363683   VR
62      15.156075   VR
63      29.650318   VR
64       0.631050   VR
65      19.793102   VR

[66 rows x 2 columns]


In [ ]:
pc = df.query('Type=="FS"')["Normalised HR"].tolist()
vr = df.query('Type=="VR"')["Normalised HR"].tolist()

normalancy_pc = stats.shapiro(pc)
normalancy_vr = stats.shapiro(vr)
print(f"FS: {normalancy_pc}")
print(f"VR: {normalancy_vr}")

result = stats.ttest_rel(pc, vr)
print(result)

print(f'Cohen-d: {cohen_d(pc, vr):.2f}')
# print(f'Cliffs: {cliffs_delta(pc, vr):.2f}')

FS: ShapiroResult(statistic=0.9776724576950073, pvalue=0.7138577699661255)
VR: ShapiroResult(statistic=0.9812292456626892, pvalue=0.8226732611656189)
TtestResult(statistic=-2.536701331638034, pvalue=0.016266536026586027, df=32)
Cohen-d: -0.41
Cliffs: 0.40


#LF/HF Value

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
lf=[[[], [], [], []],  [[], [], [], []],  [[], [], [], []]]
hf=[[[], [], [], []],  [[], [], [], []],  [[], [], [], []]]
i=0
for path in paths:
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/IBI.csv")[0]
    print(p)
    try:
      ibis = flirt.reader.empatica.read_ibi_file_into_df(p)
      hrv_features = flirt.get_hrv_features(ibis['ibi'])
      if(len(hrv_features.columns)>5):
        lf[0][i].append(hrv_features['hrv_lf'])
        hf[0][i].append(hrv_features['hrv_hf'])
    except:
      print("error")
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/IBI.csv")[0]
    print(p)
    try:
      ibis = flirt.reader.empatica.read_ibi_file_into_df(p)
      hrv_features = flirt.get_hrv_features(ibis['ibi'])
      if(len(hrv_features.columns)>5):
        lf[1][i].append(hrv_features['hrv_lf'])
        hf[1][i].append(hrv_features['hrv_hf'])
    except:
      print("error")
  i+=1

In [ ]:
data = [[[], []], [[], []], [[], []]]
i=0
for x in lf[:-1]:
  j=0
  for y in x:
    tmp=[]
    k=0
    for z in y:
      tmp.append(np.mean(z.dropna().tolist()))
      k+=1
    data[0][i].append(tmp)
  i+=1
i=0
for x in hf[:-1]:
  j=0
  for y in x:
    tmp=[]
    for z in y:
      tmp.append(np.mean(z.dropna().tolist()))
    data[1][i].append(tmp)
    j+=1
  i+=1

for i in [0, 1]:
  for x,y in zip(data[0][i], data[1][i]):
    data[2][i]+=(np.array(np.array(x)/np.array(y))).tolist()


print("LF/HF")
vr = data[2][1][:25]
pc = data[2][0][:25]
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f}-{np.percentile(pc, 75):.2f})")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f}-{np.percentile(vr, 75):.2f})")
print(f"FS: $\mu: $ {np.mean(pc):.2f}, $\sigma: $: {np.std(pc):.2f}")
print(f"VR: $\mu: $ {np.mean(vr):.2f}, $\sigma: $: {np.std(vr):.2f}")
print(stats.shapiro(pc))
print(stats.shapiro(vr))
result = stats.ttest_rel(pc, vr)
print(result)
print(cohen_d(pc, vr))

LF/HF
FS: Mdn: 1.43 (0.93-1.67)
VR: Mdn: 1.30 (0.86-1.91)
FS: $\mu: $ 1.37, $\sigma: $: 0.60
VR: $\mu: $ 1.43, $\sigma: $: 0.63
ShapiroResult(statistic=0.9474433064460754, pvalue=0.21942037343978882)
ShapiroResult(statistic=0.9661115407943726, pvalue=0.5489051342010498)
TtestResult(statistic=-0.3355687120697309, pvalue=0.7401118967486352, df=24)
-0.09446979083037069


#EDA

In [ ]:
#@title cvxEDA Algorith
"""
______________________________________________________________________________

 File:                         cvxEDA.py
 Last revised:                 07 Nov 2015 r69
 ______________________________________________________________________________

 Copyright (C) 2014-2015 Luca Citi, Alberto Greco

 This program is free software; you can redistribute it and/or modify it under
 the terms of the GNU General Public License as published by the Free Software
 Foundation; either version 3 of the License, or (at your option) any later
 version.

 This program is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
 FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

 You may contact the author by e-mail (lciti@ieee.org).
 ______________________________________________________________________________

 This method was first proposed in:
 A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
 "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
 IEEE Transactions on Biomedical Engineering, 2015
 DOI: 10.1109/TBME.2015.2474131

 If you use this program in support of published research, please include a
 citation of the reference above. If you use this code in a software package,
 please explicitly inform the end users of this copyright notice and ask them
 to cite the reference above in their published research.
 ______________________________________________________________________________
"""

import numpy as np
import cvxopt as cv
import cvxopt.solvers

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing

    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).


    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters

    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C],
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return [r, p, t, l, d, e, obj]

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
eda_PC = []
eda_VR = []
eda_BS = [[], [], []]
for path in paths:
  for participant in glob(path + "/*/Baseline", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_BS[0].append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])
    eda_BS[1].append(np.mean(flirt.reader.empatica.read_eda_file_into_df(p)["eda"]))
    eda_BS[2].append(np.std(flirt.reader.empatica.read_eda_file_into_df(p)["eda"]))
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_PC.append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_VR.append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])

In [ ]:
# 0: Phasic, SCR
# 1: Tonic, SCL
cvx_eda_PC = [[], [], [], []]
cvx_eda_VR = [[], [], [], []]
cvx_eda_BS = [[], [], [], []]
# 0 - 8  : Phasmophobia
# 8 - 15 : Warthunder
# 15 - 24: Minecraft
# 24 - 33: Dirt Rally
i=0
j=0
for eda in eda_BS[0]:
  cvx = cvxEDA(eda, 4)
  cvx_eda_BS[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1
i=0
j=0
for eda in eda_PC:
  cvx = cvxEDA(eda, 4)
  cvx_eda_PC[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1

i=0
j=0
for eda in eda_VR:
  cvx = cvxEDA(eda, 4)
  cvx_eda_VR[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1

     pcost       dcost       gap    pres   dres
 0: -5.8626e+04 -5.8625e+04  1e+03  4e+01  9e-03
 1: -5.8626e+04 -5.8732e+04  1e+02  3e+00  7e-04
 2: -5.8626e+04 -5.8634e+04  9e+00  2e-01  5e-05
 3: -5.8626e+04 -5.8627e+04  1e+00  9e-03  2e-06
 4: -5.8626e+04 -5.8627e+04  2e-01  2e-03  4e-07
 5: -5.8627e+04 -5.8627e+04  5e-02  2e-04  4e-08
 6: -5.8627e+04 -5.8627e+04  1e-02  2e-05  4e-09
 7: -5.8627e+04 -5.8627e+04  2e-03  3e-06  7e-10
 8: -5.8627e+04 -5.8627e+04  5e-04  5e-07  1e-10
 9: -5.8627e+04 -5.8627e+04  9e-05  6e-08  1e-11
10: -5.8627e+04 -5.8627e+04  7e-06  3e-09  6e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.7132e+03 -2.7120e+03  2e+03  4e+01  4e-02
 1: -2.7121e+03 -2.8051e+03  1e+02  2e+00  2e-03
 2: -2.7121e+03 -2.7181e+03  6e+00  1e-01  1e-04
 3: -2.7125e+03 -2.7133e+03  8e-01  1e-03  1e-06
 4: -2.7131e+03 -2.7133e+03  2e-01  2e-04  2e-07
 5: -2.7132e+03 -2.7132e+03  3e-02  3e-05  3e-08
 6: -2.7132e+03 -2.7132e+03  6e-03  4e-06  4e-0

In [ ]:
scr_PC = []
for x in cvx_eda_PC:
  for y in x:
    for z in y[0][100:-100]:
      scr_PC.append(z)

scr_VR = []
for x in cvx_eda_VR:
  for y in x:
    for z in y[0][100:-100]:
      scr_VR.append(z)

data = [scr_PC, scr_VR]
m1=np.median(data[0])
m2=np.median(data[1])
s1=np.percentile(data[0], 25)
s11=np.percentile(data[0], 75)
s22=np.percentile(data[1], 75)
s2=np.percentile(data[1], 25)

print(f"PC: Mdn: {m1:.2f} ({s1:.2f}-{s11:.2f})")
print(np.mean(data[0]))
print(f"VR: Mdn: {m2:.2f} ({s2:.2f}-{s22:.2f})")
print(np.mean(data[1]))

PC: Mdn: 0.00 (0.00-0.01)
0.4890472803282551
VR: Mdn: 0.00 (0.00-0.12)
0.36968008500566246


In [ ]:
# 0: Phasic, SCR
# 1: Tonic, SCL
# 0 - 8  : Phasmophobia
# 8 - 15 : Warthunder
# 15 - 24: Minecraft
# 24 - 33: Dirt Rally

scr_mean = [[], []]
tmp=[]
i=0
j=0
for x in cvx_eda_PC:
  j=0
  for y in x:
    tmp.append((np.mean(y[0]) - np.mean(cvx_eda_BS[i][j][0])) / np.std(cvx_eda_BS[i][j][0]))
    j+=1
  i+=1

print(tmp)
scr_mean[0] = tmp
tmp=[]
i=0
j=0
for x in cvx_eda_VR:
  j=0
  for y in x:
    tmp.append((np.mean(y[0]) - np.mean(cvx_eda_BS[i][j][0])) / np.std(cvx_eda_BS[i][j][0]))
    j+=1
  i+=1
scr_mean[1] = tmp

pc = scr_mean[0]
vr = scr_mean[1]
print("SCR")

data = [pc, vr]
m1=np.median(data[0])
m2=np.median(data[1])
s1=np.percentile(data[0], 25)
s11=np.percentile(data[0], 75)
s22=np.percentile(data[1], 75)
s2=np.percentile(data[1], 25)

print(len(vr))
print(len(pc))

print(f"PC: Mdn: {m1:.2f} ({s1:.2f}-{s11:.2f})")
# print(np.mean(data[0]))
print(f"VR: Mdn: {m2:.2f} ({s2:.2f}-{s22:.2f})")
# print(np.mean(data[1]))

print(f"FS: $\mu$: {np.mean(pc):.2f}, $\sigma$: {np.std(pc):.2f}")
print(f"VR: $\mu$: {np.mean(vr):.2f}, $\sigma$: {np.std(vr):.2f}")

normalancy_pc = stats.shapiro(pc)
normalancy_vr = stats.shapiro(vr)
print(f"FS: {normalancy_pc}")
print(f"VR: {normalancy_vr}")

result = stats.wilcoxon(pc, vr)
print(result)
print(f'Cohen-d: {cohen_d(pc, vr):.2f}')
print(f'Cliff\'s: {cliffs_delta(pc, vr):.2f}')

[8.908308374320173, 0.012544785355730913, 3.145025204039363, 6.656392564204403, 0.0, 6.791807161656111, 0.0, -0.004079743122753311, 0.008973075646150458, -0.0045098615555116284, -0.5668127296638737, 0.047500887728772714, 1.6804046300336268, 0.011173551186930822, 0.0, 0.021993142204224093, 27.547798161319644, 0.1304648456860656, 0.22591009072554347, 0.06689496942505993, 2.552390866050881, -0.3713069320939142, -0.4837596653797841, -0.5902047339634644, 25.417615770392977, -0.046012846176576125, 31.42282144524508, 0.11881972041940722, -0.012900176511875678, 0.717085592322033, 0.2623425717045436, -0.03727724183216073, -0.06989480107247704]
SCR
33
33
PC: Mdn: 0.02 (-0.00-1.68)
VR: Mdn: 0.45 (-0.03-3.54)
FS: $\mu$: 3.44, $\sigma$: 8.15
VR: $\mu$: 8.33, $\sigma$: 21.57
FS: ShapiroResult(statistic=0.5019033551216125, pvalue=1.8914303456796233e-09)
VR: ShapiroResult(statistic=0.43542730808258057, pvalue=3.778093393691506e-10)
WilcoxonResult(statistic=221.0, pvalue=0.29599490854889154)
Cohen-d: -

#SAM

In [ ]:
file = open('SAM.csv')

type(file)
csvreader = csv.reader(file)
sam = []
for row in csvreader:
  # print(row)
  sam.append(row)
Games=['LVLA', 'LVHA', 'HVHA', 'HVLA']

sam = pd.DataFrame(data=sam[1:],columns=sam[0])
# print(sam.dtypes)
sam["Arousal"] = sam["Arousal"].astype(str).astype(float)
sam["Valence"] = sam["Valence"].astype(str).astype(float)
sam["Dominance"] = sam["Dominance"].astype(str).astype(float)
sam["CMA Quadrant"] = sam["Game"].astype(str)
print(sam)

   Participant ID  Game  Arousal  Valence  Dominance Type CMA Quadrant
0               1  HVHA      4.0      4.0        3.0   FS         HVHA
1               2  HVHA      3.0      5.0        1.0   VR         HVHA
2               3  HVHA      4.0      3.0        2.0   FS         HVHA
3               4  HVHA      5.0      5.0        3.0   VR         HVHA
4               5  HVHA      4.0      4.0        3.0   VR         HVHA
..            ...   ...      ...      ...        ...  ...          ...
61             62  LVLA      2.0      2.0        5.0   VR         LVLA
62             63  LVLA      3.0      4.0        3.0   FS         LVLA
63             64  LVLA      3.0      4.0        4.0   FS         LVLA
64             65  LVLA      2.0      2.0        4.0   VR         LVLA
65             66  LVLA      4.0      4.0        3.0   VR         LVLA

[66 rows x 7 columns]


In [ ]:
d = sam.query('Game=="HVHA" and Type=="FS"')["Arousal"]
print(d.tolist())

[4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0]


In [ ]:
for e in ["Arousal", "Valence", "Dominance"]:
  print(e)
  for g in Games:
    print(g)
    s = sam.query(f'Game=="{g}"')
    for x in ["FS", "VR"]:
      print(x)
      lst = s.query(f'Type=="{x}"')[e]
      lower_quartile = np.percentile(lst, 25)
      median = np.median(lst)
      upper_quartile = np.percentile(lst, 75)
      iqr = upper_quartile-lower_quartile
      lower_whisker = min(lst)
      upper_whisker = max(lst)

      print("lower whisker= ", lower_whisker, ",")
      print("\t\tlower quartile= ", lower_quartile, ",")
      print("\t\tmedian= ", median, ",")
      print("\t\tupper quartile= ", upper_quartile, ",")
      print("\t\tupper whisker= ", upper_whisker)
      print()

Arousal
LVLA
FS
lower whisker=  2.0 ,
		lower quartile=  2.0 ,
		median=  2.0 ,
		upper quartile=  3.0 ,
		upper whisker=  3.0

VR
lower whisker=  1.0 ,
		lower quartile=  1.0 ,
		median=  2.0 ,
		upper quartile=  2.0 ,
		upper whisker=  4.0

LVHA
FS
lower whisker=  2.0 ,
		lower quartile=  2.75 ,
		median=  3.0 ,
		upper quartile=  3.25 ,
		upper whisker=  5.0

VR
lower whisker=  3.0 ,
		lower quartile=  3.75 ,
		median=  5.0 ,
		upper quartile=  5.0 ,
		upper whisker=  5.0

HVHA
FS
lower whisker=  4.0 ,
		lower quartile=  4.0 ,
		median=  4.0 ,
		upper quartile=  4.0 ,
		upper whisker=  5.0

VR
lower whisker=  3.0 ,
		lower quartile=  4.0 ,
		median=  5.0 ,
		upper quartile=  5.0 ,
		upper whisker=  5.0

HVLA
FS
lower whisker=  1.0 ,
		lower quartile=  2.0 ,
		median=  2.0 ,
		upper quartile=  2.5 ,
		upper whisker=  3.0

VR
lower whisker=  3.0 ,
		lower quartile=  3.0 ,
		median=  4.0 ,
		upper quartile=  4.5 ,
		upper whisker=  5.0

Valence
LVLA
FS
lower whisker=  2.0 ,
		lower qua

In [ ]:
print(sam.corr(method="spearman"))
print(stats.spearmanr(sam["Dominance"], sam["Arousal"]))
print(stats.spearmanr(sam["Arousal"], sam["Valence"]))
print(stats.spearmanr(sam["Valence"], sam["Dominance"]))

print('\n\nPC')
sampc = sam.query("Type == 'FS'")
print(sampc.corr(method="spearman"))
print(stats.spearmanr(sampc["Dominance"], sampc["Arousal"]))
print(stats.spearmanr(sampc["Arousal"], sampc["Valence"]))
print(stats.spearmanr(sampc["Valence"], sampc["Dominance"]))

print('\n\nVR')
samvr = sam.query("Type == 'VR'")
print(samvr.corr(method="spearman"))
print(stats.spearmanr(samvr["Dominance"], samvr["Arousal"]))
print(stats.spearmanr(samvr["Arousal"], samvr["Valence"]))
print(stats.spearmanr(samvr["Valence"], samvr["Dominance"]))

            Arousal   Valence  Dominance
Arousal    1.000000  0.126549  -0.268202
Valence    0.126549  1.000000  -0.265739
Dominance -0.268202 -0.265739   1.000000
SignificanceResult(statistic=-0.2682024408657573, pvalue=0.029456929164310513)
SignificanceResult(statistic=0.12654853995635715, pvalue=0.311289608891442)
SignificanceResult(statistic=-0.2657391336847788, pvalue=0.031039645374892098)


PC
            Arousal   Valence  Dominance
Arousal    1.000000 -0.140817   0.002557
Valence   -0.140817  1.000000  -0.062518
Dominance  0.002557 -0.062518   1.000000
SignificanceResult(statistic=0.0025573214201303927, pvalue=0.9887308611797079)
SignificanceResult(statistic=-0.14081714905255238, pvalue=0.4344211488265298)
SignificanceResult(statistic=-0.06251768235314005, pvalue=0.7296227553924928)


VR
            Arousal   Valence  Dominance
Arousal    1.000000  0.200469  -0.311006
Valence    0.200469  1.000000  -0.348336
Dominance -0.311006 -0.348336   1.000000
SignificanceResult(statistic=

<ipython-input-117-10d808af7c6d>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.corr(method="spearman"))
<ipython-input-117-10d808af7c6d>:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sampc.corr(method="spearman"))
<ipython-input-117-10d808af7c6d>:15: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(samvr.corr(method="spearman"))


In [ ]:
print("D>=4")
print(len(sam.query("Dominance>3")))
print(sam.query("Dominance>3").corr(method="spearman"))
print(stats.spearmanr(sam.query("Dominance>3")["Valence"], sam.query("Dominance>3")["Dominance"]))

print("FS")
print(sam.query("Dominance>3 and Type == 'FS'").corr(method="spearman"))
print(len(sam.query("Dominance>3 and Type=='FS'")["Dominance"]))
print(stats.spearmanr(sam.query("Dominance>3 and Type=='FS'")["Valence"], sam.query("Dominance>3  and Type=='FS'")["Dominance"]))
print("VR")
print(sam.query("Dominance>3 and Type == 'VR'").corr(method="spearman"))
print(len(sam.query("Dominance>3 and Type=='VR'")["Dominance"]))
print(stats.spearmanr(sam.query("Dominance>3 and Type=='VR'")["Valence"], sam.query("Dominance>3  and Type=='VR'")["Dominance"]))


print("\nD<=2")
print(len(sam.query("Dominance<3")))
print(sam.query("Dominance<3").corr(method="spearman"))
print(stats.spearmanr(sam.query("Dominance<3")["Valence"], sam.query("Dominance<3")["Dominance"]))


print("FS")
print(sam.query("Dominance<3 and Type == 'FS'").corr(method="spearman"))
print(len(sam.query("Dominance<3 and Type=='FS'")["Dominance"]))
print(stats.spearmanr(sam.query("Dominance<3 and Type=='FS'")["Valence"], sam.query("Dominance<3  and Type=='FS'")["Dominance"]))
print("VR")
print(sam.query("Dominance<3 and Type == 'VR'").corr(method="spearman"))
print(len(sam.query("Dominance<3 and Type=='VR'")["Dominance"]))
print(stats.spearmanr(sam.query("Dominance<3 and Type=='VR'")["Valence"], sam.query("Dominance<3  and Type=='VR'")["Dominance"]))


D>=4
14
            Arousal   Valence  Dominance
Arousal    1.000000  0.156671   0.160226
Valence    0.156671  1.000000  -0.427793
Dominance  0.160226 -0.427793   1.000000
SignificanceResult(statistic=-0.42779263194649864, pvalue=0.12703970384058888)
FS
            Arousal   Valence  Dominance
Arousal    1.000000 -0.525485   0.582772
Valence   -0.525485  1.000000  -0.427121
Dominance  0.582772 -0.427121   1.000000
9
SignificanceResult(statistic=-0.4271210980886246, pvalue=0.2515389710699937)
VR
           Arousal   Valence  Dominance
Arousal       1.00  0.750000   0.000000
Valence       0.75  1.000000  -0.395285
Dominance     0.00 -0.395285   1.000000
5
SignificanceResult(statistic=-0.3952847075210474, pvalue=0.510140203242436)

D<=2
32
            Arousal   Valence  Dominance
Arousal    1.000000 -0.135638  -0.099711
Valence   -0.135638  1.000000  -0.530866
Dominance -0.099711 -0.530866   1.000000
SignificanceResult(statistic=-0.5308661071580622, pvalue=0.0017724320805710939)
FS
      

<ipython-input-115-a54e64557884>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance>3").corr(method="spearman"))
<ipython-input-115-a54e64557884>:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance>3 and Type == 'FS'").corr(method="spearman"))
<ipython-input-115-a54e64557884>:11: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance>3 and Type == 'VR'").corr(method="spearman"))
<ipython-input-115-a54e64557884>

In [ ]:
for g in Games:
  x = sam.query(f'Game=="{g}"')
  # print(x)
  a=x["Arousal"].tolist()
  print(f"{g} Arousal Mdn: {np.median(a):.1f} ({np.percentile(a, 25):.1f} - {np.percentile(a, 75):.1f})")
  a=x["Valence"].tolist()
  print(f"{g} Valence Mdn: {np.median(a):.1f} ({np.percentile(a, 25):.1f} - {np.percentile(a, 75):.1f})")

LVLA Arousal Mdn: 2.0 (2.0 - 2.8)
LVLA Valence Mdn: 3.0 (2.2 - 4.0)
LVHA Arousal Mdn: 3.5 (3.0 - 5.0)
LVHA Valence Mdn: 3.0 (2.0 - 4.0)
HVHA Arousal Mdn: 4.0 (4.0 - 5.0)
HVHA Valence Mdn: 4.0 (3.0 - 4.8)
HVLA Arousal Mdn: 3.0 (2.0 - 3.8)
HVLA Valence Mdn: 4.0 (3.0 - 4.0)


In [ ]:
PC = sam.query(f'Type=="FS"')
VR = sam.query(f'Type=="VR"')

print("Arousal")
print(stats.shapiro(sam['Arousal']))
pc = PC['Arousal'].tolist()
vr = VR['Arousal'].tolist()
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(len(pc))
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

print("Valence")
print(stats.shapiro(sam['Valence']))
pc = PC['Valence']
vr = VR['Valence']
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(len(pc))
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

print("Dominance")
print(stats.shapiro(sam['Dominance']))
pc = PC['Dominance']
vr = VR['Dominance']
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(len(pc))
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")


Arousal
ShapiroResult(statistic=0.897082507610321, pvalue=4.8546404286753386e-05)
ShapiroResult(statistic=0.8877468705177307, pvalue=0.0026009853463619947)
ShapiroResult(statistic=0.8563315868377686, pvalue=0.0004707006155513227)
33
FS: Mdn: 3.00 (2.00 - 4.00)
FS: $\mu = $ 3.0 $\sigma = $ 1.0
VR: Mdn: 4.00 (3.00 - 5.00)
VR: $\mu = $ 3.6 $\sigma = $ 1.3
W: 66.5 p-value: 0.007
d = -0.5261057786980897
cliff's = 0.2516069788797062
Valence
ShapiroResult(statistic=0.9005023837089539, pvalue=6.551821570610628e-05)
ShapiroResult(statistic=0.8781664967536926, pvalue=0.0015153985004872084)
ShapiroResult(statistic=0.8696938753128052, pvalue=0.000953220296651125)
33
FS: Mdn: 3.00 (3.00 - 4.00)
FS: $\mu = $ 3.3 $\sigma = $ 0.9
VR: Mdn: 4.00 (3.00 - 5.00)
VR: $\mu = $ 3.6 $\sigma = $ 1.2
W: 75.5 p-value: 0.262
d = -0.308732022447085
cliff's = 0.2773186409550046
Dominance
ShapiroResult(statistic=0.9041874408721924, pvalue=9.102379408432171e-05)
ShapiroResult(statistic=0.9030362963676453, pvalue=0.006

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [ ]:
for i in ["Arousal", "Valence", "Dominance"]:
  x=0
  print(i)
  for g in Games:
    # print(f"{g}, ")
    d = PC.query(f'Game=="{g}"')
    print(f"({x}, {np.mean(d[i]):.2f}) +- (0, {np.std(d[i])/2:.2f})")
    x+=1
  x=0
  print()
  for g in Games:
    # print(f"{g}, ")
    d = VR.query(f'Game=="{g}"')
    print(f"({x}, {np.mean(d[i]):.2f}) +- (0, {np.std(d[i])/2:.2f})")
    #  +- (0, {np.std(PC["Arousal"])/2:.2f})")
    x+=1
  print()

Arousal
(0, 2.33) +- (0, 0.24)
(1, 3.12) +- (0, 0.46)
(2, 4.22) +- (0, 0.21)
(3, 2.14) +- (0, 0.32)

(0, 2.00) +- (0, 0.47)
(1, 4.38) +- (0, 0.43)
(2, 4.44) +- (0, 0.34)
(3, 3.86) +- (0, 0.42)

Valence
(0, 3.56) +- (0, 0.48)
(1, 2.88) +- (0, 0.39)
(2, 3.22) +- (0, 0.39)
(3, 3.57) +- (0, 0.45)

(0, 3.00) +- (0, 0.62)
(1, 3.38) +- (0, 0.50)
(2, 4.22) +- (0, 0.61)
(3, 4.00) +- (0, 0.38)

Dominance
(0, 2.67) +- (0, 0.41)
(1, 3.50) +- (0, 0.43)
(2, 2.44) +- (0, 0.42)
(3, 2.86) +- (0, 0.49)

(0, 3.00) +- (0, 0.67)
(1, 2.25) +- (0, 0.33)
(2, 2.11) +- (0, 0.55)
(3, 1.57) +- (0, 0.36)



In [ ]:
for g in Games:
  print(g)
  for t in ["VR", "FS", "All"]:
    print(t)
    if(t!="All"):
      x = sam.query(f'Game=="{g}" and Type=="{t}"')
      a=x['Arousal']
      v=x['Valence']
      r=np.sqrt(np.square(x['Arousal'].tolist()) + np.square(x['Valence'].tolist()))
      # print(f'& {np.mean(a):.2f} & {np.std(a):.2f} & {np.mean(v):.2f} & {np.std(v):.2f} & {np.mean(r):.2f} & {np.std(r):.2f} \\\\')
      print(f'& {np.mean(a):.2f} & {np.std(a):.2f} & {np.mean(v):.2f} & {np.std(v):.2f} & {np.mean(r):.2f} & {np.std(r):.2f} \\\\')
    else:
      x = sam.query(f'Game=="{g}"')
      a=x['Arousal']
      v=x['Valence']
      r=np.sqrt(np.square(x['Arousal'].tolist()) + np.square(x['Valence'].tolist()))
      print(f'& {np.mean(a):.2f} & {np.std(a):.2f} & {np.mean(v):.2f} & {np.std(v):.2f} & {np.mean(r):.2f} & {np.std(r):.2f} \\\\')

LVLA
VR
& 2.00 & 0.94 & 3.00 & 1.25 & 3.72 & 1.25 \\
FS
& 2.33 & 0.47 & 3.56 & 0.96 & 4.27 & 0.98 \\
All
& 2.17 & 0.76 & 3.28 & 1.15 & 4.00 & 1.16 \\
LVHA
VR
& 4.38 & 0.86 & 3.38 & 0.99 & 5.61 & 0.87 \\
FS
& 3.12 & 0.93 & 2.88 & 0.78 & 4.36 & 0.70 \\
All
& 3.75 & 1.09 & 3.12 & 0.93 & 4.99 & 1.01 \\
HVHA
VR
& 4.44 & 0.68 & 4.22 & 1.23 & 6.25 & 0.69 \\
FS
& 4.22 & 0.42 & 3.22 & 0.79 & 5.36 & 0.53 \\
All
& 4.33 & 0.58 & 3.72 & 1.15 & 5.81 & 0.76 \\
HVLA
VR
& 3.86 & 0.83 & 4.00 & 0.76 & 5.62 & 0.72 \\
FS
& 2.14 & 0.64 & 3.57 & 0.90 & 4.24 & 0.79 \\
All
& 3.00 & 1.13 & 3.79 & 0.86 & 4.93 & 1.02 \\


In [ ]:
x = sam.query(f'Type=="VR"')
y = sam.query(f'Type=="FS"')
rx=np.sqrt(np.square(x['Arousal'].tolist()) + np.square(x['Valence'].tolist()))
ry=np.sqrt(np.square(y['Arousal'].tolist()) + np.square(y['Valence'].tolist()))
print(f'VR: $\mu$ {np.mean(rx):.2f}, $\sigma$ {np.std(rx):.2f}')
print(f'FS: $\mu$ {np.mean(ry):.2f}, $\sigma$ {np.std(ry):.2f}')
print(stats.shapiro(rx))
print(stats.shapiro(ry))
r = stats.wilcoxon(rx, ry)
print(r)
print(f"d = {cohen_d(rx, ry): .2f}")
print(f"cliff's = {cliffs_delta(rx, ry):.2f}")

VR: $\mu$ 5.27, $\sigma$ 1.35
FS: $\mu$ 4.58, $\sigma$ 0.90
ShapiroResult(statistic=0.9074306488037109, pvalue=0.008349988609552383)
ShapiroResult(statistic=0.9564009308815002, pvalue=0.204054594039917)
WilcoxonResult(statistic=92.5, pvalue=0.0039764433100765335)
d =  0.59
cliff's = 0.55


In [ ]:
print("Arousal")
pc = PC['Arousal'].tolist()
vr = VR['Arousal'].tolist()
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}\n")

print("Valence")
pc = PC['Valence']
vr = VR['Valence']
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}\n")

print("Dominance")
pc = PC['Dominance']
vr = VR['Dominance']
print(stats.shapiro(pc))
print(stats.shapiro(vr))
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

Arousal
ShapiroResult(statistic=0.8877468705177307, pvalue=0.0026009853463619947)
ShapiroResult(statistic=0.8563315868377686, pvalue=0.0004707006155513227)
FS: Mdn: 3.00 (2.00 - 4.00)
FS: $\mu = $ 3.0 $\sigma = $ 1.0
VR: Mdn: 4.00 (3.00 - 5.00)
VR: $\mu = $ 3.6 $\sigma = $ 1.3
W: 66.5 p-value: 0.007
d = -0.5261057786980897
cliff's = 0.2516069788797062

Valence
ShapiroResult(statistic=0.8781664967536926, pvalue=0.0015153985004872084)
ShapiroResult(statistic=0.8696938753128052, pvalue=0.000953220296651125)
FS: Mdn: 3.00 (3.00 - 4.00)
FS: $\mu = $ 3.3 $\sigma = $ 0.9
VR: Mdn: 4.00 (3.00 - 5.00)
VR: $\mu = $ 3.6 $\sigma = $ 1.2
W: 75.5 p-value: 0.262
d = -0.308732022447085
cliff's = 0.2773186409550046

Dominance
ShapiroResult(statistic=0.9030362963676453, pvalue=0.006392336450517178)
ShapiroResult(statistic=0.8744556903839111, pvalue=0.0012349631870165467)
FS: Mdn: 3.00 (2.00 - 4.00)
FS: $\mu = $ 2.8 $\sigma = $ 1.0
VR: Mdn: 2.00 (1.00 - 3.00)
VR: $\mu = $ 2.3 $\sigma = $ 1.1
W: 88.0 p-val

In [ ]:
import scikit_posthocs as sp
# PC-VR-Mix
Games=['LVLA', 'LVHA', 'HVHA', 'HVLA']
arousal=[[], [], []]
valence=[[], [], []]

bygame=[[], [], [], []]
bygame[0] = sam[sam['Game'] == 'LVLA']
bygame[1] = sam[sam['Game'] == 'LVHA']
bygame[2] = sam[sam['Game'] == 'HVHA']
bygame[3] = sam[sam['Game'] == 'HVLA']

for (i, df) in enumerate(bygame):
  # print(Games[i])

  # Mix
  # print("")
  ar = df['Arousal'].tolist()
  vl = df['Valence'].tolist()
  # print(f"{np.mean(ar)} {np.std(ar)}")
  # print(f"{np.mean(vl)} {np.std(vl)}")
  arousal[2].append(ar)
  valence[2].append(vl)

  ar = df.query('Type=="FS"')['Arousal'].tolist()
  vl = df.query('Type=="FS"')['Valence'].tolist()
  arousal[0].append(ar)
  valence[0].append(vl)

  ar = df.query('Type=="VR"')['Arousal'].tolist()
  vl = df.query('Type=="VR"')['Valence'].tolist()
  arousal[1].append(ar)
  valence[1].append(vl)

for i in range(2,3):
  print(["PC", "VR", "All"][i])
  print("Arousal")
  result = stats.kruskal(arousal[i][0], arousal[i][1], arousal[i][2], arousal[i][3])
  print(result)


  print("Valence")
  result = stats.kruskal(valence[i][0], valence[i][1], valence[i][2], valence[i][3])
  print(result)
  print()
  # for j in range(4):
  #   print(Games[j])
  #   print(f"Arousal: M {np.mean(arousal[i][j]):0.2f}, SD {np.std(arousal[i][j]):0.2f}")
  #   print(f"Valence: M {np.mean(valence[i][j]):0.2f}, SD {np.std(valence[i][j]):0.2f}")

  print("Post hoc: ")
  adf = sam[['Arousal', 'CMA Quadrant']].copy()
  print("\nArousal")
  ph = sp.posthoc_dunn(sam, group_col='CMA Quadrant', val_col='Arousal')
  print(ph)
  print("\nValence")
  ph = sp.posthoc_dunn(sam, group_col='CMA Quadrant', val_col='Arousal')
  print(ph)

All
Arousal
KruskalResult(statistic=30.671288942014424, pvalue=9.96883973452592e-07)
Valence
KruskalResult(statistic=4.75557827232003, pvalue=0.19059511595420348)

Post hoc: 

Arousal
              HVHA      HVLA      LVHA          LVLA
HVHA  1.000000e+00  0.002271  0.156609  1.364451e-07
HVLA  2.271275e-03  1.000000  0.100571  6.047019e-02
LVHA  1.566091e-01  0.100571  1.000000  2.189760e-04
LVLA  1.364451e-07  0.060470  0.000219  1.000000e+00

Valence
              HVHA      HVLA      LVHA          LVLA
HVHA  1.000000e+00  0.002271  0.156609  1.364451e-07
HVLA  2.271275e-03  1.000000  0.100571  6.047019e-02
LVHA  1.566091e-01  0.100571  1.000000  2.189760e-04
LVLA  1.364451e-07  0.060470  0.000219  1.000000e+00


#VAS

In [ ]:
file = open('VAS.csv')

type(file)
csvreader = csv.reader(file)
vas = []
for row in csvreader:
  # print(row)
  vas.append(row)

vas = pd.DataFrame(data=vas[1:],columns=vas[0])
vas["CMA Quadrant"] = vas["Game"].astype(str)

In [ ]:
columns = ['Joy', 'Anger', 'Calmness', 'Sadness', 'Disgust', 'Relaxation', 'Happiness', 'Fear', 'Anxiousness', 'Dizziness']

for c in columns:
  vas[c] = vas[c].astype(str).astype(float)
print(vas.dtypes)
print(vas.head())

Id               object
Game             object
Dropdown         object
Joy             float64
Anger           float64
Calmness        float64
Sadness         float64
Disgust         float64
Relaxation      float64
Happiness       float64
Fear            float64
Anxiousness     float64
Dizziness       float64
CMA Quadrant     object
dtype: object
  Id  Game Dropdown   Joy  Anger  Calmness  Sadness  Disgust  Relaxation  \
0  1  HVHA       VR  81.0   14.0      35.0     16.0     59.0        53.0   
1  2  HVHA       FS  75.0    0.0      54.0     39.0     33.0        52.0   
2  3  HVHA       FS  69.0    0.0      28.0      0.0      0.0         7.0   
3  4  HVHA       VR  71.0    0.0       4.0      0.0     15.0         0.0   
4  5  HVHA       FS  28.0    0.0      54.0      0.0     46.0        32.0   

   Happiness  Fear  Anxiousness  Dizziness CMA Quadrant  
0       85.0  16.0         16.0        0.0         HVHA  
1       86.0   6.0         16.0        0.0         HVHA  
2       52.0   0.0 

In [ ]:
arr=[]
columns = ['Joy', 'Anger', 'Calmness', 'Sadness', 'Disgust', 'Relaxation', 'Happiness', 'Fear', 'Anxiousness', 'Dizziness']
games = ["HVHA", "HVLA", "LVHA", "LVLA"]
PC = vas.query('Dropdown=="FS"')
VR = vas.query('Dropdown=="VR"')

for g in games:
  # print(g)
  pcg = PC[PC['Game'] == g]
  vrg = VR[VR['Game'] == g]
  for c in columns:
    # print(c)
    df = pd.DataFrame(pcg[c].tolist(), columns = ["Val"])
    df["Game"] = g
    df["Emotion"] = c
    df["Type"] = "FS"
    arr.append(df)
    df = pd.DataFrame(vrg[c].tolist(), columns = ["Val"])
    df["Game"] = g
    df["Emotion"] = c
    df["Type"] = "VR"
    arr.append(df)
    # print(c)

df =  pd.concat(arr, ignore_index=True)
print(df)

      Val  Game    Emotion Type
0    75.0  HVHA        Joy   FS
1    69.0  HVHA        Joy   FS
2    28.0  HVHA        Joy   FS
3    47.0  HVHA        Joy   FS
4    50.0  HVHA        Joy   FS
..    ...   ...        ...  ...
655  38.0  LVLA  Dizziness   VR
656  49.0  LVLA  Dizziness   VR
657  65.0  LVLA  Dizziness   VR
658  17.0  LVLA  Dizziness   VR
659  30.0  LVLA  Dizziness   VR

[660 rows x 4 columns]


In [ ]:
for e in columns:
  print(e)
  print("FS")
  fs = PC[e]
  lower_quartile = np.percentile(fs, 25)
  median = np.median(fs)
  upper_quartile = np.percentile(fs, 75)
  iqr = upper_quartile-lower_quartile
  lower_whisker = max(0, lower_quartile-(1.5*iqr))
  upper_whisker = min(upper_quartile+(1.5*iqr), 100)

  print("lower whisker= ", lower_whisker, ",")
  print("\t\tlower quartile= ", lower_quartile, ",")
  print("\t\tmedian= ", median, ",")
  print("\t\tupper quartile= ", upper_quartile, ",")
  print("\t\tupper whisker= ", upper_whisker)
  print()

  print("VR")
  vr = VR[e]
  lower_quartile = np.percentile(vr, 25)
  median = np.median(vr)
  upper_quartile = np.percentile(vr, 75)
  iqr = upper_quartile-lower_quartile
  lower_whisker = max(0, lower_quartile-(1.5*iqr))
  upper_whisker = min(upper_quartile+(1.5*iqr), 100)

  print("lower whisker= ", lower_whisker, ",")
  print("\t\tlower quartile= ", lower_quartile, ",")
  print("\t\tmedian= ", median, ",")
  print("\t\tupper quartile= ", upper_quartile, ",")
  print("\t\tupper whisker= ", upper_whisker)
  print()

Joy
FS
lower whisker=  0 ,
		lower quartile=  28.0 ,
		median=  55.0 ,
		upper quartile=  70.0 ,
		upper whisker=  100

VR
lower whisker=  0 ,
		lower quartile=  50.0 ,
		median=  70.0 ,
		upper quartile=  85.0 ,
		upper whisker=  100

Anger
FS
lower whisker=  0 ,
		lower quartile=  0.0 ,
		median=  10.0 ,
		upper quartile=  26.0 ,
		upper whisker=  65.0

VR
lower whisker=  0 ,
		lower quartile=  2.0 ,
		median=  18.0 ,
		upper quartile=  40.0 ,
		upper whisker=  97.0

Calmness
FS
lower whisker=  0 ,
		lower quartile=  0.0 ,
		median=  24.0 ,
		upper quartile=  58.0 ,
		upper whisker=  100

VR
lower whisker=  0 ,
		lower quartile=  1.0 ,
		median=  11.0 ,
		upper quartile=  50.0 ,
		upper whisker=  100

Sadness
FS
lower whisker=  0 ,
		lower quartile=  0.0 ,
		median=  0.0 ,
		upper quartile=  19.0 ,
		upper whisker=  47.5

VR
lower whisker=  0 ,
		lower quartile=  0.0 ,
		median=  0.0 ,
		upper quartile=  10.0 ,
		upper whisker=  25.0

Disgust
FS
lower whisker=  0 ,
		lower quartile= 

In [ ]:
for e in columns:
  # print(e)
  d = df.query(f'Emotion=="{e}"')
  x = d.query('Type=="VR"')["Val"].tolist()
  y = d.query('Type=="FS"')["Val"].tolist()
  result=stats.wilcoxon(x, y)
  print(f"{e}: W: {result[0]:.2f}, p-value: {result[1]:.4f}")
  # print(f"d = {cohen_d(x, y): .2f}")
  print(f"Cliffs = {cliffs_delta(x, y): .2f}\n")

Joy: W: 101.50, p-value: 0.0041
Cliffs =  0.51

Anger: W: 171.50, p-value: 0.3198
Cliffs =  0.46

Calmness: W: 107.50, p-value: 0.1389
Cliffs =  0.43

Sadness: W: 48.00, p-value: 0.3007
Cliffs =  0.26

Disgust: W: 50.50, p-value: 0.2176
Cliffs =  0.37

Relaxation: W: 93.00, p-value: 0.1708
Cliffs =  0.35

Happiness: W: 174.00, p-value: 0.1469
Cliffs =  0.43

Fear: W: 184.00, p-value: 0.2096
Cliffs =  0.47

Anxiousness: W: 260.50, p-value: 0.9478
Cliffs =  0.49

Dizziness: W: 8.50, p-value: 0.0000
Cliffs =  0.53



/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
